In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install seqeval datasets allennlp
!pip install flax
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 10.1 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 65.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 346 kB 12.3 MB/s 
     |████████████████████████████████| 719 kB 50.6 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 140 kB 53.0 MB/s 
     |████████████████████████████████| 212 kB 44.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.6 MB/s 
     |███████████████

In [3]:
%cd /content/drive/MyDrive/Bert_Lime/CFG/

/content/drive/.shortcut-targets-by-id/1CLIDh5rSBKopppgMh7zb8C3_AWNoz6B8/Bert_Lime/CFG


In [4]:
!ls

Bert  data.pkl	Longformer  Transformer  untitled


In [5]:
import os
from operator import itemgetter    
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

# from keras import models, regularizers, layers, optimizers, losses, metrics
# from keras.models import Sequential
# from keras.layers import Dense
# from tensorflow.keras.utils import to_categorical
# from keras.layers import LayerNormalization
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [6]:
import random as rd

# Example Grammar rules
# 1.
# S: aSa
# S: bSb
# S: c
# 2.
# S: AcB
# A: aA | a
# B: bB | b
# 3.
# S: AcB
# A: aA | a | bA | b
# B: bB | b | aB | a

rules = [
        {
        "S":[ 
            ["a", "S", "a"] ,
            ["b", "S", "b"] ,
            ["S_bar"],
            ["S_bar"]
        ], 
        "S_bar":[
            ["c"]
        ]
    }, 
        {
        "S":[ 
            ["A", "c", "B"],
            ["A", "c", "B"],
            ["A", "c", "B"]
        ], 
        "A": [
            ["a", "A"] ,
            ["A_bar"],
            ["A_bar"]
        ],
        "A_bar": [
            ["a"]
        ],
        "B": [
            ["b", "B"] ,
            ["B_bar"],
            ["B_bar"]
        ],
        "B_bar": [
            ["b"]
        ]
    },
        {
        "S":[ 
            ["A", "c", "B"],
            ["A", "c", "B"],
            ["A", "c", "B"]
        ], 
        "A": [
            ["a", "A"] ,
            ["b", "A"],
            ["A_bar"] ,
            ["A_bar"]
        ],
        "A_bar": [
            ["a"],
            ["b"]
        ],
        "B": [
            ["a", "B"] ,
            ["b", "B"] ,
            ["B_bar"],
            ["B_bar"]
        ],
        "B_bar": [
            ["a"],
            ["b"]
        ]
    }

]


# Contributed by Tiger Sachase
# Used to parse any list of strings and insert them in place in a list 
def generate_items(items):
    for item in items:
        if isinstance(item, list):
            for subitem in generate_items(item):
                yield subitem
        else:
            yield item       

# Our expansion algo
def expansion(start, len_max, last_ind, ind_rule):
    for element in start:
        if element in rules[ind_rule]:
            loc = start.index(element)
            # print(start)
            if(len(start) > len_max):
                start[loc] = rd.choice(rules[ind_rule][element])
            else:
                start[loc] = rd.choice(rules[ind_rule][element][:-last_ind])
        result = [item for item in generate_items(start)]
    
    for item in result:
        if not isinstance(item, list):
            if item in rules[ind_rule]:
                result = expansion(result, len_max, last_ind, ind_rule)
    
    return result


def to_string(result):
    return ' '.join(result)

from tqdm import tqdm
def generate_dataset(total_size, len_of_cfg):
    
    train_data = []
    train_label = []
    test_data = []
    test_label = []

    for i in tqdm(range(total_size)):
        for j in range(3):
            train_data.append(to_string(expansion(["S"], len_of_cfg, 2, j)))
            train_label.append(j)
            test_data.append(to_string(expansion(["S"], len_of_cfg, 2, j)))
            test_label.append(j)
    temp =  list(zip(train_data, train_label))
    rd.shuffle(temp)
    train_data, train_label = zip(*temp)
    train_data, train_label = list(train_data), list(train_label)

    temp =  list(zip(test_data, test_label))
    rd.shuffle(temp)
    test_data, test_label = zip(*temp)
    test_data, test_label = list(test_data), list(test_label)
    

    return (train_data, train_label), (test_data, test_label)

# # An example test you can run to see it at work
# result = ["S"]
# print(result) # Print our starting result
# result = expansion(result, 200, 2, 2) # Expand our starting list 
# final = to_string(result)


In [7]:
import datetime
a = datetime.datetime.now()
b = datetime.datetime.now()
print((b - a).total_seconds())

3.7e-05


In [8]:

# (train_data, train_label), (test_data, test_label) = generate_dataset(5000, 200)

In [9]:
import pickle

In [10]:
%ls

Bert/  data.pkl  Longformer/  Transformer/  untitled


In [11]:
data_path = "data.pkl"


In [12]:

# with open(data_path, "wb") as f:
#   pickle.dump(zip((train_data, train_label), (test_data, test_label)), f)

In [13]:
with open(data_path, "rb") as f:
   (train_data, train_label), (test_data, test_label) = zip(*pickle.load(f))
   (train_data, train_label), (test_data, test_label) = (list(train_data), list(train_label)), (list(test_data), list(test_label))

In [14]:
# print(train_data)
print(len(train_label))

15000


In [15]:
import torch

In [16]:
print(torch.cuda.is_available())

False


In [17]:
model_version = 'allenai/longformer-base-4096'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [18]:
ac_dict = {"class_1" : 0, "class_2" : 1, "class_3": 2}

In [19]:
import torch.nn as nn
import torch
# !torch.cuda.clear_cache

/bin/bash: torch.cuda.clear_cache: command not found


In [20]:
from huggingface_hub import create_repo

In [21]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class MyTransformer(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, n_class: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model = d_model, vocab_size = ntoken, dropout = dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.classifier = nn.Linear(d_model, n_class)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.classifier.bias.data.zero_()
        self.classifier.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        # print(src.shape, src_mask.shape)
        # print(src)
        src_mask = src_mask.bool()
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = torch.mean(output, dim = 1)
        output = self.classifier(output)
        # print(output.shape)
        return output

In [22]:
class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        print(vocab_size, d_model)
        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [23]:
ntokens = 1000  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 5  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 5  # number of heads in nn.MultiheadAttention
dropout = 0.1  # dropout probability
n_classes = 3
from transformers import LongformerConfig, LongformerTokenizer, LongformerModel

tokenizer = LongformerTokenizer.from_pretrained(model_version,
                                          bos_token = "[CLS]",
                                          eos_token = "[SEP]")
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12).to(device)
cross_entropy_layer = nn.CrossEntropyLoss()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
device

device(type='cpu')

In [25]:
from tqdm import tqdm

In [26]:
from typing import List, Tuple

In [27]:
def pad_batch(elems: List[List[int]], pad_token_id: int) -> List[List[int]]:
    """Pads all lists in elems to the maximum list length of any list in 
    elems. Pads with pad_token_id.
    """
    max_len = max([len(elem) for elem in elems])
    return [elem+[pad_token_id]*(max_len-len(elem)) for elem in elems]

def load_dataset(tokenizer):
  with open(data_path, "rb") as f:
    (train_data, train_labels), (test_data, test_labels) = zip(*pickle.load(f))
    (train_data, train_labels), (test_data, test_labels) = (list(train_data), list(train_labels)), (list(test_data), list(test_labels))

  new_train_data = []
  for tokenized_text in tqdm(train_data):
    new_encoded_text = tokenizer.encode(tokenized_text)
    new_train_data.append(new_encoded_text)
  
  new_test_data = []
  for tokenized_text in tqdm(test_data):
    new_encoded_text = tokenizer.encode(tokenized_text)
    new_test_data.append(new_encoded_text)
    # print(new_encoded_text)
  return (new_train_data, train_labels), (new_test_data, test_labels)

def generator(dataset_data, dataset_label, max_len = 512, batch_size = 16):
  i = 0
  tokenized_threads, labels = [], []
  while i<len(dataset_data):
    tokenized_threads.append(dataset_data[i][:max_len])
    labels.append(dataset_label[i])
    i += 1
        
    if i%batch_size==0:
      yield (pad_batch(tokenized_threads, tokenizer.pad_token_id), 
                  labels)
      tokenized_threads, labels = [], []

In [28]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters()),
                       lr = 2e-5)

In [29]:
(train_dataset, train_labels), (test_dataset, test_labels) = load_dataset(tokenizer)

100%|██████████| 15000/15000 [00:23<00:00, 648.18it/s]


In [30]:
import datetime
train_loss_list = []
test_loss_list = []
stats_list = []

In [31]:
def train(dataset_data, dataset_label, batch_size):
    global values_weight, values_bias;
    global train_loss_list, stats_list;
    accumulate_over = 32
    
    optimizer.zero_grad()
    print("Training")
    for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label,  batch_size = batch_size)):
        a = datetime.datetime.now()
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        labels = torch.tensor(labels, device=device, dtype=torch.long)


        loss = compute((tokenized_threads, 
                        labels,), False)
        b = datetime.datetime.now()
        print("\rLoss: ", loss.item(), "| time taken:", (b - a).total_seconds(), end = " ")

        loss.backward()

        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
        train_loss_list.append(loss.item())
            
    optimizer.step()

In [32]:
import datasets

In [33]:

from datasets import load_metric
metric1 = load_metric("accuracy")
metric3 = load_metric("precision", average = "micro")
metric2 = load_metric("f1", average = "micro")
metric4 = load_metric("recall", average = "micro")
metric5 = load_metric("precision", average = "macro")
metric6 = load_metric("f1", average = "macro")
metric7 = load_metric("recall", average = "macro")


In [34]:
def evaluate(dataset_data, dataset_label, BATCH_SIZE):
    global test_loss_list, stats_list;
    int_to_labels = {v:k for k, v in ac_dict.items()}
    print('Evaluation')
    
    with torch.no_grad():
        for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE)):
            # print(comp_type_labels)
            #Cast to PyTorch tensor
            tokenized_threads = torch.tensor(tokenized_threads, device=device)
            labels = torch.tensor(labels, device=device)

            preds = compute((tokenized_threads, 
                            labels,), pred=True)
            loss = compute((tokenized_threads, labels,), pred = False)
            
            metric1.add_batch(predictions=preds, 
                            references=labels,)
                            #tokenized_threads=tokenized_threads.cpu().tolist())
            metric2.add_batch(predictions=preds, 
                            references=labels,)
            metric3.add_batch(predictions=preds, 
                            references=labels,)
            metric4.add_batch(predictions=preds, 
                            references=labels,)
            metric5.add_batch(predictions=preds, 
                            references=labels,)
            metric6.add_batch(predictions=preds, 
                            references=labels,)
            metric7.add_batch(predictions=preds, 
                            references=labels,)
            
            print("\rLoss: ", loss.item(), end = " ")
            test_loss_list.append(loss.item())

    print()
    stats = []
    stats.append(metric1.compute())
    stats.append(metric5.compute(average = "macro"))
    stats.append(metric6.compute(average = "macro"))
    stats.append(metric7.compute(average = "macro"))
    print(stats[0])
    print("Micro")
    print(metric2.compute(average = "micro"))
    print(metric3.compute(average = "micro"))
    print(metric4.compute(average = "micro"))
    print("Macro")
    print(stats[1])
    print(stats[2])
    print(stats[3])
    stats_list.append(stats)

In [35]:

(train_dataset_data, train_dataset_label), (test_dataset_data, test_dataset_label) = load_dataset(tokenizer)

100%|██████████| 15000/15000 [00:16<00:00, 920.79it/s]


In [36]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor], pred: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, labels = batch
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, torch.tensor(1).to(device), torch.tensor(0).to(device))
    logits = transformer_model(tokenized_threads, pad_mask)
    print(logits)


    if(pred):
      return torch.argmax(logits, dim = 1)


    ce_loss = cross_entropy_layer(logits, labels)

    return ce_loss

In [ ]:
import numpy as np
def shuffle(data, labels):
  idx = np.random.permutation(len(data))
  x,y = np.array(data)[idx], np.array(labels)[idx]
  return x.tolist(), y.tolist()

In [ ]:
BATCH_SIZE = 256

In [ ]:
print(tokenizer.decode([101]))

 like


In [ ]:
print(device)

cpu


In [ ]:
linear_path = "Transformer/linear_layer_64.pt"
cross_path = "Transformer/cross_entropy_layer_64.pt"
tokenizer_path = "Transformer/tokenizer_pre_64.pkl"
transformer_path = "Transformer/transformer_layer_64.pt"
data_path_x = "Transformer/data_64.pkl"

In [ ]:
# torch.save(linear_layer.state_dict(), linear_path)
# torch.save(cross_entropy_layer.state_dict(), cross_path)
# torch.save(transformer_model.state_dict(), transformer_path)
# with open(data_path, "wb") as f:
#   pickle.dump([train_loss_list, test_loss_list, stats_list], f)
# with open(tokenizer_path, "wb") as f:
#   pickle.dump(tokenizer, f)

In [ ]:
n_epochs = 4
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset_data, train_dataset_label = shuffle(train_dataset_data, train_dataset_label)
    test_dataset_data, test_dataset_label = shuffle(test_dataset_data, test_dataset_label)
    X_train, Y_train = train_dataset_data[:], train_dataset_label[:]
    X_test, Y_test = test_dataset_data[:], test_dataset_label[:]
    train(X_train, Y_train, BATCH_SIZE)
    print("Test accuracy")
    evaluate(X_test, Y_test, BATCH_SIZE)

------------EPOCH 1---------------
Training
Loss:  1.1795252561569214 | time taken: 8.092961 Test accuracy
Evaluation
Loss:  1.2048612833023071 
{'accuracy': 0.3330414870689655}
Micro
{'f1': 0.3330414870689655}
{'precision': 0.3330414870689655}
{'recall': 0.3330414870689655}
Macro
{'precision': 0.11101382902298851}
{'f1': 0.1665572003570286}
{'recall': 0.3333333333333333}
------------EPOCH 2---------------
Training
Loss:  1.2313053607940674 | time taken: 7.991541 Test accuracy
Evaluation
Loss:  1.1969162225723267 

In [ ]:
print([train_loss_list, test_loss_list, stats_list])

[[], [], []]


In [ ]:
# test_dataset_data, test_dataset_label = shuffle(test_dataset_data[:100],  test_dataset_label[:100])
# evaluate(test_dataset_data, test_dataset_label, BATCH_SIZE)
  

In [ ]:
print(len(test_dataset_label))

In [ ]:
print(test_dataset_label)

In [ ]:
import pickle

In [ ]:
torch.save(linear_layer.state_dict(), linear_path)
torch.save(cross_entropy_layer.state_dict(), cross_path)
torch.save(transformer_model.state_dict(), transformer_path)
with open(data_path_x, "wb") as f:
  pickle.dump([train_loss_list, test_loss_list, stats_list], f)
with open(tokenizer_path, "wb") as f:
  pickle.dump(tokenizer, f)

In [ ]:
linear_layer.load_state_dict(torch.load(linear_path, map_location=device))
cross_entropy_layer.load_state_dict(torch.load(cross_path, map_location=device))
transformer_model.load_state_dict(torch.load(transformer_path , map_location=device))
with open(tokenizer_path, "rb") as f:
  tokenizer = pickle.load(f)

In [ ]:
from multiprocessing import Process, Manager, Array
manager = Manager()

In [ ]:
graph_attention = []
for i in range(12):
  graph_attention.append({})
mapping_ind = {}
rev_mapping_ind = {}

In [ ]:
cnt = 0
threshold = 0.01

In [ ]:
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:


my_arr = [1 for i in range(100)]

def func(i, j , k):
    global my_arr
    my_arr[i] = 3
    

indexes = [(i, i, i) for i in range(12)]
pool = mp.Pool(min(12, mp.cpu_count()))
pool.starmap(func, indexes)
pool.close()

In [ ]:
print(my_arr)

In [ ]:
type(mp.cpu_count())

In [ ]:
temp_attention = []
def init_arr(arr):
    globals()['arr'] = arr
from copy import deepcopy
def store_attention(lay, j, tokens):
        
    global mapping_ind, graph_attention, temp_attention;
    maximum_attention = np.max(temp_attention)
    weights = temp_attention[j, lay, :, :]
    temp_dict = {}
    for ind_1 in range(len(tokens)):
        for ind_2 in range(len(tokens)):
            if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                continue
            node_1 = mapping_ind[tokens[ind_1]]
            node_2 = mapping_ind[tokens[ind_2]]
            weight = weights[ind_1, ind_2]
            if(weight < threshold*maximum_attention):
                continue
            if(node_1 not in temp_dict):
                temp_dict[node_1] = {}
            
            if(node_2  not in temp_dict[node_1]):
                temp_dict[node_1][node_2] = (weight, 1)
            else:
                temp_dict[node_1][node_2] = (weight*temp_dict[node_1][node_2][1]/(temp_dict[node_1][node_2][1] + 1), temp_dict[node_1][node_2][1] + 1)
    graph_attention[lay] = deepcopy(temp_dict)
#     print(graph_attention)
def attention_graph(dataset_data, dataset_label, BATCH_SIZE):
    
    
    accumulate_over = 4
    global cnt, temp_attention, manager;
    optimizer.zero_grad()

    for i, (tokenized_threads, labels) in tqdm(enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE))):
    
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        temp_attention = attention[:, :, :, :]
        maximum_attention = np.max(attention)
        for j, tokenized_thread in enumerate(tokenized_threads):
            tokens = tokenizer.convert_ids_to_tokens(tokenized_thread) 
            print(len(tokens))
            print(tokenized_thread.shape)
            for tok in tokens:
                if(tok not in mapping_ind):
                    mapping_ind[tok] = cnt;
                    cnt += 1
            
    for i, (tokenized_threads, labels) in tqdm(enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE))):
    
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        temp_attention = attention[:, :, :, :]
        maximum_attention = np.max(attention)
        for j, tokenized_thread in enumerate(tokenized_threads):
            tokens = tokenizer.convert_ids_to_tokens(tokenized_thread) 

#             params = [(i1, j, tokens) for i1 in range(12)]
#             pool = mp.Pool(min(12, mp.cpu_count()))
#             pool.starmap(store_attention, params)
#             pool.close()
            for lay in range(12):
                weights = attention[j, lay, :, :]
                for ind_1 in range(len(tokens)):
                    for ind_2 in range(len(tokens)):
                        if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                            continue

                        node_1 = mapping_ind[tokens[ind_1]]
                        node_2 = mapping_ind[tokens[ind_2]]
                        weight = weights[ind_1, ind_2]
                        if(weight < threshold*maximum_attention):
                            continue
                        if(node_1 not in graph_attention[lay]):
                            graph_attention[lay][node_1] = {}
                        if(node_2  not in graph_attention[lay][node_1]):
                            graph_attention[lay][node_1][node_2] = (weight, 1)
                        else:
                            graph_attention[lay][node_1][node_2] = ((weight + (graph_attention[lay][node_1][node_2][0])*graph_attention[lay][node_1][node_2][1])/(graph_attention[lay][node_1][node_2][1] + 1), graph_attention[lay][node_1][node_2][1] + 1)
#         print(cnt, cnt**2)              


In [ ]:
def extract_data(dataset_data, dataset_label, BATCH_SIZE):
    example = [[], []]
    for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE)):
        for j in range(BATCH_SIZE):
            if(len(example[0]) < 5 or len(example[1]) < 5):
                example[labels[j]].append((tokenizer.convert_ids_to_tokens(tokenized_threads[j]), tokenized_threads[j]))
            else:
                break
    return example

In [ ]:
text = extract_data(test_dataset_data, test_dataset_label, BATCH_SIZE)

In [ ]:
vocab = [reverse_word_index[i] for i in reverse_word_index]
import random

In [ ]:
print(vocab)
print(len(vocab))

In [ ]:
def find_topk_tokens(graph, k):
    attention = {}
    attention_inward = {}
    for node_1 in graph:
        if(node_1 not in attention):
            attention[node_1] = (0, 0)
        for node_2 in graph[node_1]:
            
            if(node_2 not in attention_inward):
                attention_inward[node_2] = (0, 0)
            attention[node_1] = ((graph[node_1][node_2][0] + (attention[node_1][0])*attention[node_1][1])/(attention[node_1][1] + 1), attention[node_1][1] + 1)
            attention_inward[node_2] = ((graph[node_1][node_2][0] + (attention_inward[node_2][0])*attention_inward[node_2][1])/(attention_inward[node_2][1] + 1), attention_inward[node_2][1] + 1)
    out_max = [i[0] for i in find_maxk(attention, k)]
    in_max = [i[0] for i in find_maxk(attention_inward, k)]
    return in_max, out_max
def find_maxk(attention, k):
    ret = []
    for i in attention:
        if(i == mapping_ind["[CLS]"] or i == mapping_ind["[SEP]"]):
            continue
        ret.append((i, attention[i][0]))
    ret.sort(key = lambda x: x[1])
    ret.reverse()
    return ret[:k]

In [ ]:
def remove_pad(given_s):
    l = given_s.split()
    ret = []
    for i in l:
        if(i != "[PAD]"):
            ret.append(i)
    return " ".join(ret)

In [ ]:
def replacing(tok_mapping, tokenized_thread, tokens):
    ret_tokenized_thread = tokenized_thread.clone()
    ind = []
    for i in range(len(tokens)):
        if(mapping_ind[tokens[i]] in tok_mapping):
            ret_tokenized_thread[0, i] = tokenizer.encode(tok_mapping[mapping_ind[tokens[i]]])[1];
            ind.append(i)
    return ret_tokenized_thread, ind

In [ ]:
import random
def create_mapping(tok, ind):
    ret_dict = {}
    for k, i in enumerate(tok):
        if((k + i) < len(vocab)):
            ret_dict[i] = vocab[k + ind]
        else:
            ret_dict[i] = random.choice(vocab)
    return ret_dict

In [ ]:
def get_embedding(ind, tokenized):
    pad_mask = torch.where(tokenized!=tokenizer.pad_token_id, 1, 0)
    embedding = transformer_model(input_ids=tokenized,attention_mask=pad_mask,).last_hidden_state
    ret_embedding = []
    for i in ind:
        ret_embedding.append(embedding[:, i, :].reshape((embedding.shape[2])))
    return ret_embedding

In [ ]:
replacement = 100
trials = 1
trial_candidate = 5
select_max = 5

In [ ]:
print(len(text[1]))

In [ ]:
import matplotlib.pyplot as plt
from scipy import spatial
text[0] = text[0][:3]
text[1] = text[1][:3]

In [ ]:
def predict_proba(tokenized_threads):
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,attention_mask=pad_mask,).last_hidden_state[:, 0, :])
    logits = torch.exp(logits)
    z = torch.sum(logits)
    logits = torch.divide(logits, z)
    return logits
def mask_out_top5(text_lab, label, label_ind, flag):
    accumulate_over = 4
    global cnt, temp_attention, manager;
    optimizer.zero_grad()
    for i, (tokens, tokenized_threads) in enumerate(text_lab):
        print("\n\n\n")
        print("###### Example :", str(i), "label", label, "########")
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device).reshape((1, len(tokenized_threads)))
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
        
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        graph = dict()
        
        for tok in tokens:
            if(tok not in mapping_ind):
                mapping_ind[tok] = cnt;
                cnt += 1

        for lay in range(12):
            weights = attention[0, lay, :, :]
            for ind_1 in range(len(tokens)):
                for ind_2 in range(len(tokens)):
                    if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                        continue
                    node_1 = mapping_ind[tokens[ind_1]]
                    node_2 = mapping_ind[tokens[ind_2]]
                    rev_mapping_ind[node_1] = tokens[ind_1]
                    rev_mapping_ind[node_2] = tokens[ind_2]
                    weight = weights[ind_1, ind_2]
                    if(node_1 not in graph):
                        graph[node_1] = {}
                    if(node_2  not in graph[node_1]):
                        graph[node_1][node_2] = (weight, 1)
                    else:
                        graph[node_1][node_2] = ((weight + (graph[node_1][node_2][0])*graph[node_1][node_2][1])/(graph[node_1][node_2][1] + 1), graph[node_1][node_2][1] + 1)

        topk_in, topk_out = find_topk_tokens(graph, select_max)
        
        candidate = random.sample(topk_in, trial_candidate)
        for j in range(trials):
            random.shuffle(vocab)
            print([ rev_mapping_ind[z] for z in candidate], "\n")
            in_logits_l2_norm = []
            for l in tqdm(range(0, min(len(vocab), 20000), select_max)):
                top5_in_mapping = create_mapping(candidate, l)

                example_in, ind_in = replacing(top5_in_mapping, tokenized_threads, tokens)

                actual_in = get_embedding(ind_in, tokenized_threads)
                replaced_in = get_embedding(ind_in, example_in)

                if(len(ind_in) >= 1):
                    final_actual_in = torch.divide(sum(actual_in),len(actual_in)).cpu().detach().numpy()
                    final_replaced_in = torch.divide(sum(replaced_in) , len(replaced_in)).cpu().detach().numpy()
                else:
                    continue
                if(predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind] < 0.5):
                    continue
                subtract_in = np.subtract(final_actual_in, final_replaced_in)
                if(flag == "L2"):
                    in_logits_l2_norm.append((np.linalg.norm(subtract_in), predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind]))
                elif(flag == "Cosine"):
                    in_logits_l2_norm.append((spatial.distance.cosine(final_actual_in, final_replaced_in), predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind]))
                    
                    
            in_logits_l2_norm.sort(key = lambda x : x[0])
            

            plt.scatter([i[0] for i in in_logits_l2_norm],[i[1] for i in in_logits_l2_norm])
        plt.title('Example ' + str(i) + ' in')
        plt.xlabel(flag + ' norm')
        plt.ylabel('probability of '+ label )
        plt.show()
        plt.clf()
        
        candidate = random.sample(topk_out, trial_candidate)


        for j in range(trials):
            random.shuffle(vocab)
            print([ rev_mapping_ind[z] for z in candidate], "\n")
            out_logits_l2_norm = []
            for l in tqdm(range(0, min(len(vocab), 20000), select_max)):
                top5_out_mapping = create_mapping([j], l)

                
                
                example_out, ind_out = replacing(top5_out_mapping, tokenized_threads, tokens)

                actual_out = get_embedding(ind_out, tokenized_threads)
                replaced_out = get_embedding(ind_out, example_out)
#                 print(sum(actual_out))
                if(len(ind_out) >= 1):
                    final_actual_out = torch.divide(sum(actual_out) , len(actual_out)).cpu().detach().numpy()
                    final_replaced_out = torch.divide(sum(replaced_out) , len(replaced_out)).cpu().detach().numpy()
                else:
                    continue
                if(predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind] < 0.50):
                    continue
                subtract_out = np.subtract(final_actual_out, final_replaced_out)
                if(flag == "L2"):
                    out_logits_l2_norm.append((np.linalg.norm(subtract_out), predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind]))
                elif(flag == "Cosine"):
                    out_logits_l2_norm.append((spatial.distance.cosine(final_actual_out, final_replaced_out), predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind]))
                    
            out_logits_l2_norm.sort(key = lambda x : x[0])
            
            plt.scatter([i[0] for i in out_logits_l2_norm],[i[1] for i in out_logits_l2_norm])

        plt.title('Example ' + str(i) + ' out')
        plt.xlabel(flag + ' norm')
        plt.ylabel('probability of ' + label )
        plt.show()
        plt.clf()

In [ ]:

mask_out_top5(text[0], "negative", 0, "L2")


In [ ]:
mask_out_top5(text[1], "positive", 1, "L2",)

In [ ]:
mask_out_top5(text[0], "negative", 0, "Cosine")

In [ ]:
mask_out_top5(text[1], "positive", 1, "Cosine",)

In [ ]:
print(graph_attention)

In [ ]:
attention_graph(train_dataset_data[:100], train_dataset_label[:100], BATCH_SIZE)

In [ ]:
def remove_padding(x):
    ret = []
    for i in x[::-1]:
        if(i == "[PAD]"):
            continue
        else:
            ret.append(i)
    ret = ret[::-1]
    return ret

In [ ]:
for i, sent in enumerate(text):
    text[i] = remove_padding(sent)


In [ ]:
color = ["\033[1;31;47m", "\033[1;32;47m", "\033[1;33;47m", 
         "\033[1;34;47m", "\033[1;35;47m", "\033[1;36;47m",
         "\033[1;37;47m"]
mapping = ["Red", "Green", "Yellow", "Blue", "Purple", "Cyan", "White"]


In [ ]:
print("\033[1;32;47m" + text[0][0])

In [ ]:
reverse_mapping = {}
for i in mapping_ind:
    reverse_mapping[mapping_ind[i]] = i

In [ ]:
tokens = []
for lay in range(12):
    for i in graph_attention[lay]:
        l = 0
        for j in graph_attention[lay][i]:
            l += graph_attention[lay][i][j][0]
        l /= len(graph_attention[lay][i])
        tokens.append((l, reverse_mapping[i]))


In [ ]:
tokens_high_deg = []
for lay in range(12):
    for i in graph_attention[lay]:
        tokens_high_deg.append((len(graph_attention[lay][i]), reverse_mapping[i]))

In [ ]:
tokens.sort(reverse = True)
tokens_high_deg.sort(reverse = True)

In [ ]:
print(tokens[:10])
print(tokens_high_deg[:10])

In [ ]:
factor = 40
factor_2 = 20

In [ ]:
Truetop_tok = [i[1] for i in tokens[:len(mapping)*factor]]
top_tok_deg = [i[1] for i in tokens_high_deg[:len(mapping)*factor_2]]


In [ ]:
def parse_top_k(l, length):
    ret = []
    ind = 0
    mapping_check = {}
    while(len(ret) != length and ind < len(l)):
        if(l[ind][1] in mapping_check):
            ind += 1
            continue
        mapping_check[l[ind][1]] = 1
        ret.append(l[ind][1])
        ind += 1
    return ret
top_tok = parse_top_k(tokens, len(mapping)*factor)

top_tok_deg = parse_top_k(tokens_high_deg, len(mapping)*factor_2)

In [ ]:
print("Attention")
print(top_tok)
print("Degree ")
print(top_tok_deg)

In [ ]:
print(len(top_tok_deg))

In [ ]:
print(set(top_tok))


In [ ]:
print(set(top_tok_deg))

In [ ]:
def masking_text(tok, text, color):
    for i in range(len(text)):
        print("\t ###################### Example " + str(i) + " ##############")
        print("")
        masked_text = []
        for j in text[i]:
            flag = 0
            for ind in range(len(tok)):
                if(j == tok[ind]):
                    masked_text.append(color[ind//factor] + j + "\x1b[0m")
                    flag = 1
                    break
            if(flag == 0):
                masked_text.append(j)
        print(" ".join(masked_text))   
        
        print("")

In [ ]:
print("\t ####### Mapping for high attention value \n \n")
for i in range(len(mapping)):
    print(color[i], mapping[i], " : ", top_tok[i*factor : (i + 1)*factor])

In [ ]:
for i in range(len(mapping)):
    print(mapping[i], " : ", "(attention value) priority top : ", str(i*factor), "-", str((i + 1)*factor))

In [ ]:
print(masking_text(top_tok, text, color))

In [ ]:
print("\t ####### Mapping for Degree value \n \n")
for i in range(len(mapping)):
    print(color[i] ,mapping[i], " : ", top_tok_deg[i*factor_2 : (i + 1)*factor_2])

In [ ]:
for i in range(len(mapping)):
    print(mapping[i], " : ", "(degree) priority top : ", str(i*factor), "-", str((i + 1)*factor_2))

In [ ]:
print(masking_text(top_tok_deg, text, color))

In [ ]:
print(set(i[1] for i in tokens[:100]))

In [ ]:
print(set(i[1] for i in tokens_high_deg[:200]))

In [ ]:
import matplotlib.pyplot as plt

def plot_degree_dist(degrees):
    plt.hist(degrees)
    plt.show()


In [ ]:
for lay in range(12):
    degree = []
    for i in graph_attention[lay]:
        degree.append(len(graph_attention[lay][i]))
    degree.sort(reverse = True)
    plot_degree_dist(degree)
    